In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50     # steepness factor
num_blocks = 10000 # number of expression blocks
num_data = 9100    # number of data
test_size = 100    # number of test size
val_size = 1000    # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
growth_rate = 1.05              # growth rate of penalty weight
warmup = 100                    # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, growth_rate, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Iters 0, Validation Loss: 2509939.02
Epoch 0, Iters 125, Validation Loss: 23547784.62
Epoch 1, Iters 250, Validation Loss: 58453234.75
Epoch 2, Iters 375, Validation Loss: 82014416.00
Epoch 3, Iters 500, Validation Loss: 1776728.77
Epoch 4, Iters 625, Validation Loss: 2318687.25
Epoch 5, Iters 750, Validation Loss: 1643472.52
Epoch 6, Iters 875, Validation Loss: 750567.14
Epoch 7, Iters 1000, Validation Loss: 900527072.00
Epoch 8, Iters 1125, Validation Loss: 910287.37
Epoch 9, Iters 1250, Validation Loss: 1253404.10
Epoch 10, Iters 1375, Validation Loss: 952552.71
Epoch 11, Iters 1500, Validation Loss: 858877.54
Epoch 12, Iters 1625, Validation Loss: 824673.31
Epoch 13, Iters 1750, Validation Loss: 174276908032.00
Epoch 14, Iters 1875, Validation Loss: 573625.55
Epoch 15, Iters 2000, Validation Loss: 783659.38
Epoch 16, Iters 2125, Validation Loss: 847936.79
Epoch 17, Iters 2250, Validation Loss: 823323.28
Epoch 18, Iters 2375, Validation Loss: 807027.33
Epoch 19, Iters 2500,

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-10000_g.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:18<00:00,  1.39s/it]


             Obj Val  Constraints Viol  Elapsed Time
count     100.000000        100.000000    100.000000
mean   142102.980120       1785.941804      0.044634
std     38637.936368       3820.048675      0.005131
min     83062.975146          0.000000      0.010794
25%    110094.680678          0.000000      0.043946
50%    147255.797306          0.000000      0.044629
75%    161496.426811         83.878990      0.045814
max    230706.941452      17946.592166      0.051339
Number of infeasible solution: 25


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
growth_rate = 1.05              # growth rate of penalty weight
warmup = 100                    # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, growth_rate, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Iters 0, Validation Loss: 2261953.14
Epoch 0, Iters 125, Validation Loss: 5502742.38
Epoch 1, Iters 250, Validation Loss: 17991258.31
Epoch 2, Iters 375, Validation Loss: 4205453.38
Epoch 3, Iters 500, Validation Loss: 836171.16
Epoch 4, Iters 625, Validation Loss: 6899973.69
Epoch 5, Iters 750, Validation Loss: 12085742.38
Epoch 6, Iters 875, Validation Loss: 1298501.09
Epoch 7, Iters 1000, Validation Loss: 1087976.26
Epoch 8, Iters 1125, Validation Loss: 921761.50
Epoch 9, Iters 1250, Validation Loss: 1376663.16
Epoch 10, Iters 1375, Validation Loss: 612936.82
Epoch 11, Iters 1500, Validation Loss: 762866.61
Epoch 12, Iters 1625, Validation Loss: 488358.23
Epoch 13, Iters 1750, Validation Loss: 671314.54
Epoch 14, Iters 1875, Validation Loss: 1021050.08
Epoch 15, Iters 2000, Validation Loss: 1672443.76
Epoch 16, Iters 2125, Validation Loss: 1435136.34
Epoch 17, Iters 2250, Validation Loss: 536777.08
Epoch 18, Iters 2375, Validation Loss: 503884.12
Epoch 19, Iters 2500, Valid

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-10000_g.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:13<00:00,  1.33s/it]


            Obj Val  Constraints Viol  Elapsed Time
count    100.000000        100.000000    100.000000
mean   60509.271532       1232.051515      0.044827
std    11539.331154       2486.978075      0.004711
min    38755.102983          0.000000      0.011948
25%    52333.009093          0.000000      0.044089
50%    57979.340427          0.000000      0.044768
75%    66492.302390       1100.669028      0.046354
max    87122.916837       9996.014389      0.050841
Number of infeasible solution: 31
